# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/I2wvg/src/ase_conversions.jl:100
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/I2wvg/src/ase_conversions.jl:100


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/I2wvg/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298698032567                   -0.85    5.4    230ms
  2   -8.300236306333       -2.81       -1.25    1.2    106ms
  3   -8.300445239383       -3.68       -1.89    2.8    131ms
  4   -8.300461657362       -4.78       -2.76    1.1    116ms
  5   -8.300464065454       -5.62       -3.07    2.6    144ms
  6   -8.300464374532       -6.51       -3.25    1.0   96.7ms
  7   -8.300464505462       -6.88       -3.40    1.0    103ms
  8   -8.300464582468       -7.11       -3.55    1.0   97.3ms
  9   -8.300464627631       -7.35       -3.74    1.0   98.1ms
 10   -8.300464632637       -8.30       -3.82    1.0   98.0ms
 11   -8.300464642542       -8.00       -4.10    1.0    102ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/I2wvg/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.63478238143                   -0.70    5.4    512ms
  2   -16.67866881105       -1.36       -1.14    1.4    224ms
  3   -16.67922445169       -3.26       -1.87    3.6    296ms
  4   -16.67927361944       -4.31       -2.67    1.2    221ms
  5   -16.67928490336       -4.95       -3.05    4.9    382ms
  6   -16.67928615632       -5.90       -3.47    1.6    236ms
  7   -16.67928620640       -7.30       -4.08    2.2    244ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/I2wvg/src/ase_conversions.jl:100
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     10
│      5
│     11
│      7
│     11
│      1
│     11
│      3
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.31254729285                   -0.56    7.4    1.60s
  2   -33.33466635813       -1.66       -0.99    1.8    761ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      5
│      4
│     10
│     14
│      1
│     11
│      3
│      3
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -33.33596232634       -2.89       -1.71    6.4    1.14s
  4   -33.33610091874       -3.86       -2.54    2.8    1.15s
  5   -33.33677134269       

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/I2wvg/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298609352880                   -0.85    5.2    228ms
  2   -8.300281471419       -2.78       -1.59    1.2   84.5ms
  3   -8.300437482766       -3.81       -2.68    3.1    125ms
  4   -8.300435144088   +   -5.63       -2.51    2.9    140ms
  5   -8.300464444002       -4.53       -3.40    1.0   80.1ms
  6   -8.300464618252       -6.76       -3.99    3.8    123ms
  7   -8.300464640830       -7.65       -4.38    2.9    134ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/I2wvg/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32894124832                   -0.56    7.0    1.46s
  2   -33.31686453778   +   -1.92       -1.27    1.5    626ms
  3   -15.39168728056   +    1.25       -0.44    6.2    1.47s
  4   -33.32448458432        1.25       -1.79    5.1    1.30s
  5   -33.24486186523   +   -1.10       -1.33    3.2    1.01s
  6   -32.69933171362   +   -0.26       -1.16    3.9    1.05s
  7   -33.33100646992       -0.20       -2.11    4.4    1.07s
  8   -33.33464952206       -2.44       -2.30    2.4    969ms
  9   -33.33659439045       -2.71       -2.45    2.6    796ms
 10   -33.33668797088       -4.03       -2.81    1.8    693ms
 11   -33.33690166168       -3.67       -3.13    2.5    798ms
 12   -33.33692952501       -4.55 

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.